## Generate Q-A pairs for Pale chapters and Generative Pseudo-labeling to help improve the performance of the DPR. This will lean on a few different sets of examples: [Generating Examples](https://haystack.deepset.ai/tutorials/18_gpl) and [Training your own model](https://haystack.deepset.ai/tutorials/09_dpr_training) being the 2 examples from Haystack, and [this](https://kaushikshakkari.medium.com/open-domain-question-answering-series-part-6-fine-tuning-dense-retriever-models-without-any-aed7c08c33ff) and [this](https://www.pinecone.io/learn/gpl/) provide some moer general content.

1. Generate QA pairs for the listed set of files (Pale). This entire set should be preprocessed in a slightly different way than they are for the primary retreiver (the number of max tokens is a bit longer than the 256 max for the DPR models used in the retreivers).

2. Save our these pais and make sure they match the formatting needed for the retreiver in the daily driver primary models.

3. Repeat this for the other novels as needed. there may be value generating these questions at the same time or there may not - unsure but not likely.

In [1]:
!nvidia-smi

Sat Feb 18 02:08:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install "faiss-gpu>=1.6.3,<2"
!pip install -q git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 KB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.6 MB/s eta 0:00:00
  

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [4]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from datasets import load_dataset


In [5]:
import os
print(os.getcwd())
os.chdir('./drive/MyDrive/pale-companion-files/GPL/')

/content


In [6]:
import pickle
with open('../worm_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

In [7]:
from haystack import Document
chapter_documents = [Document.from_dict(d) for d in chapters]
len(chapter_documents)

272

In [8]:
from haystack.nodes import PreProcessor

word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=30,
    progress_bar=True, 
    add_page_number=True
)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
corpus = word_preprocessor.process(chapter_documents)
len(corpus)

Preprocessing:   0%|          | 0/272 [00:00<?, ?docs/s]

9846

In [10]:
# We load the TAS-B model, a state-of-the-art model trained on MS MARCO
max_seq_length = 300
model_name = "msmarco-distilbert-base-tas-b"

org_model = SentenceTransformer(model_name)
org_model.max_seq_length = max_seq_length

In [11]:
print("Len Corpus:", len(corpus))

Len Corpus: 9846


In [12]:
from haystack.nodes.retriever import EmbeddingRetriever
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", similarity="cosine")
document_store.write_documents(corpus)


retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/msmarco-distilbert-base-tas-b",
    model_format="sentence_transformers",
    max_seq_len=max_seq_length,
    progress_bar=False,
)
document_store.update_embeddings(retriever)


Writing Documents:   0%|          | 0/9846 [00:00<?, ?it/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/msmarco-distilbert-base-tas-b
INFO:haystack.document_stores.faiss:Updating embeddings for 9846 docs...


Updating Embedding:   0%|          | 0/9846 [00:00<?, ? docs/s]

In [13]:
from haystack.nodes.question_generator import QuestionGenerator
from haystack.nodes.label_generator import PseudoLabelGenerator

use_question_generator = True


if use_question_generator:
    questions_producer = QuestionGenerator(
        model_name_or_path="doc2query/msmarco-t5-base-v1",
        max_length=64,
        split_length=128,
        split_overlap=30,
        batch_size=8,
        num_queries_per_doc=3,
    )

else:
    questions_producer = query_doc_pairs

# We can use either QuestionGenerator or already generated questions in PseudoLabelGenerator
psg = PseudoLabelGenerator(questions_producer, retriever, max_questions_per_document=10, batch_size=8, top_k=10)
output, pipe_id = psg.run(documents=document_store.get_all_documents())


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Using sep_token, but it is not set yet.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Generating questions:   0%|          | 0/19559 [00:00<?, ?it/s]

Mine negatives:   0%|          | 0/7231 [00:00<?, ?it/s]

Score margin:   0%|          | 0/7231 [00:00<?, ?it/s]

In [14]:
with open('./worm-gpl-output.pkl','wb') as fp:
  pickle.dump(output,fp)

In [15]:
output['gpl_labels'][9]

{'question': 'what is the difference between a task and a problem',
 'pos_doc': 'She does not feel joy at this.  This is the task.  Means to ends.\nShe is utterly blind in the present, with no eyesight or other senses to perceive things in the now.  No sight, no hearing, no touch or taste.  Not a crippling flaw, and a difficult flaw for others to use against her.  The present is only a fragment in a long span of time when one can see the past and future both.\nBut she faces an obstacle that she is utterly blind to, now.  No apparent past or future.  In interacting with it, she is limited to context.  She sees not the obstacle, but she can see things that are set in motion around it.  She cannot see it strike, but she can see the reaction, the aftermath.\nShe sees the stone fly out of the darkness, and she can determine where it was thrown from.\nThere is a task to be completed, but things must be set in place first.\nAn obstacle must be removed.  This is critical, but she is blind to i